DL model to detect if osteoarthritis is present or not (provide the probability of having osteoarthritis) in a given knee X-ray image using inceptionV3(GoogleNet) architecture.

The Dataset contains three folders: <br>
    Test (845 images)<br>
    Train (2350 images)<br>
    Valid (641 images)<br>
Each of these folders has two folders:<br>
    Test - Normal & Osteoarthritis<br>
    Train - Normal & Osteoarthritis<br>
    Valid - Normal & Osteoarthritis<br>

In [3]:
import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow import keras

In [4]:
from keras.utils import to_categorical 
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [3]:
train_dir = r'C:\Users\geetika singh\Downloads\Osteoarthritis_dataset\train'
val_dir = r'C:\Users\geetika singh\Downloads\Osteoarthritis_dataset\Valid'
test_dir = r'C:\Users\geetika singh\Downloads\Osteoarthritis_dataset\test'

In [ ]:
keras.backend.clear_session()
del model

### Image Preprocessing using OpenCV

In [4]:
x_train=[]
x_val=[]
x_test=[]
categories = ["Normal","Osteoarthritis"]

In [5]:
num=0
for category in categories:
    path = os.path.join(train_dir, category)
    for img in os.listdir(path):
        img_arr = cv.imread(os.path.join(path,img),cv.IMREAD_GRAYSCALE)
        img_arr = cv.resize(img_arr,(300,224))
        img_arr = cv.equalizeHist(img_arr)
        #segmentation (cropping) & equalisation (contour)
        #img_arr = cv.equalizeHist(img_arr[150:350,50:450])
        #DenseNet169 needs 3 channel input
        #img_arr = cv.cvtColor(img_arr, cv.COLOR_GRAY2RGB)
        x_train.append(img_arr)
        num+=1

In [6]:
num=0
for category in categories:
    path = os.path.join(val_dir, category)
    for img in os.listdir(path):
        img_arr = cv.imread(os.path.join(path,img),cv.IMREAD_GRAYSCALE)
        img_arr = cv.resize(img_arr,(300,224))
        img_arr = cv.equalizeHist(img_arr)
        #img_arr = cv.equalizeHist(img_arr[150:350,50:450])
        #img_arr = cv.cvtColor(img_arr, cv.COLOR_GRAY2RGB)
        x_val.append(img_arr)
        num+=1

In [7]:
num=0
for category in categories:
    path = os.path.join(test_dir, category)
    class_num = categories.index(category)
    for img in os.listdir(path):
        img_arr = cv.imread(os.path.join(path,img),cv.IMREAD_GRAYSCALE)
        img_arr = cv.resize(img_arr,(300,224))
        img_arr = cv.equalizeHist(img_arr)
        #img_arr = cv.equalizeHist(img_arr[150:350,50:450])
        #img_arr = cv.cvtColor(img_arr, cv.COLOR_GRAY2RGB)
        x_test.append(img_arr)
        num+=1

In [ ]:
plt.imshow(img_arr)
plt.show()

In [9]:
x_train=np.array(x_train)
x_val=np.array(x_val)
x_test=np.array(x_test)
#normalization
x_train=x_train/255.0
x_val=x_val/255.0
x_test=x_test/255.0

In [10]:
datagen = ImageDataGenerator()
train_dataset = datagen.flow_from_directory(train_dir, 
                                            target_size=(300,224),
                                            batch_size=16,
                                            color_mode='grayscale',
                                            class_mode='binary')
val_dataset = datagen.flow_from_directory(val_dir, 
                                          target_size=(300,224),
                                          batch_size=16,
                                          color_mode='grayscale',
                                          class_mode = 'binary')
test_dataset = datagen.flow_from_directory(test_dir,
                                           target_size=(300,224),
                                           batch_size=16, 
                                           color_mode='grayscale',
                                           class_mode = 'binary')

Found 1840 images belonging to 2 classes.
Found 641 images belonging to 2 classes.
Found 845 images belonging to 2 classes.


In [11]:
train_dataset.class_indices

{'Normal': 0, 'Osteoarthritis': 1}

In [12]:
y_train = train_dataset.classes
y_val = val_dataset.classes
y_test = test_dataset.classes

In [13]:
x_train.shape,y_train.shape

((1840, 224, 300), (1840,))

In [17]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224,300, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [18]:
checkpoint_cb = ModelCheckpoint("Osteoarthritis_OpenCV_Preprocessed.h5", save_best_only=True)
early_stopping_cb = EarlyStopping(monitor='val_loss', mode='min', 
                                   patience=3, restore_best_weights=True)
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience = 2, 
                                            verbose=1,factor=0.3, min_lr=0.000001)

In [19]:
history = model.fit(x_train,y_train, 
                    epochs=10, shuffle=True,
                    validation_data=(x_val,y_val), 
                    callbacks=[checkpoint_cb, early_stopping_cb, lr_reduction])

Epoch 1/10
58/58 [==============================] - 187s 3s/step - loss: 0.9858 - acc: 0.7397 - val_loss: 0.7362 - val_acc: 0.6724 - lr: 0.0010
Epoch 2/10


C:\Users\geetika singh\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


58/58 [==============================] - 214s 4s/step - loss: 0.4560 - acc: 0.7962 - val_loss: 0.6016 - val_acc: 0.7114 - lr: 0.0010
Epoch 3/10
58/58 [==============================] - 228s 4s/step - loss: 0.4226 - acc: 0.8071 - val_loss: 0.7227 - val_acc: 0.6724 - lr: 0.0010
Epoch 4/10
58/58 [==============================] - ETA: 0s - loss: 0.3689 - acc: 0.8277
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.
58/58 [==============================] - 230s 4s/step - loss: 0.3689 - acc: 0.8277 - val_loss: 0.6601 - val_acc: 0.5289 - lr: 0.0010
Epoch 5/10
58/58 [==============================] - 232s 4s/step - loss: 0.3479 - acc: 0.8283 - val_loss: 0.6906 - val_acc: 0.6708 - lr: 3.0000e-04


In [20]:
model = keras.models.load_model("Osteoarthritis_OpenCV_Preprocessed.h5")

In [21]:
y_pred=model.predict(x_test)
y_pred=np.argmax(y_pred,axis=1)
print(accuracy_score(y_pred,y_test))

27/27 [==============================] - 18s 657ms/step
0.6733727810650888


In [22]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       1.00      0.67      0.80       845
           1       0.00      0.00      0.00         0

    accuracy                           0.67       845
   macro avg       0.50      0.34      0.40       845
weighted avg       1.00      0.67      0.80       845



C:\Users\geetika singh\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\geetika singh\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\geetika singh\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
print(confusion_matrix(y_pred,y_test))

[[569 276]
 [  0   0]]


## InceptionV3

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   shear_range=0.2, 
                                   zoom_range=0.2, 
                                   horizontal_flip=True,)

train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    target_size=(500, 500), 
                                                    batch_size=16, 
                                                    class_mode='binary')

datagen = ImageDataGenerator(rescale=1./255)

validation_generator = datagen.flow_from_directory(val_dir, 
                                                   target_size=(500, 500), 
                                                   batch_size=16, 
                                                   class_mode='binary')

test_generator = datagen.flow_from_directory(test_dir, 
                                             target_size=(500, 500),
                                             batch_size=16,
                                             shuffle=False,
                                             class_mode='binary')

In [ ]:
conv_base = keras.applications.InceptionV3(weights='imagenet',
                                           include_top=False,
                                           input_shape=(500, 500, 3))

In [ ]:
model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
checkpoint_cb2 = ModelCheckpoint("Osteoarthritis_InceptionV3.h5", save_best_only=True)
early_stopping_cb2 = EarlyStopping(monitor='val_loss', mode='min', 
                                   patience=3, restore_best_weights=True)
lr_reduction2 = ReduceLROnPlateau(monitor='val_loss', patience = 2, 
                                            verbose=1,factor=0.3, min_lr=0.000001)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
weights = compute_class_weight(class_weight='balanced', 
                               classes=np.unique(train_generator.classes),
                               y=train_generator.classes)
cw = dict(zip( np.unique(train_generator.classes), weights))
print(cw)

In [ ]:
history = model.fit(train_generator, 
                    epochs = 10, 
                    validation_data = validation_generator, 
                    class_weight = cw,
                    callbacks = [checkpoint_cb2, early_stopping_cb2, lr_reduction2])

In [5]:
model = keras.models.load_model("Osteoarthritis_InceptionV3.h5")

In [ ]:
model.evaluate(test_generator)

66% accuracy on test set using InceptionV3 & Data Augmentation when run on T4 GPU (Google Collab). No Image preprocessing was done. 

try building InceptionV3 model on preprocessed images via OpenCV on Collab.